In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time

In [2]:

df = pd.read_csv('chapter_links.csv',dtype = str)

In [3]:
linklist = df['Tariff Rate URL'].tolist()

In [ ]:
def fetch_with_retry(url, retries=5, delay=5):
    """Gửi request với retry khi lỗi."""
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=60)
            response.encoding = 'utf-8'
            if response.status_code == 200:
                return response
        except requests.RequestException as e:
            print(f"Request error: {e} - Retrying {attempt+1}/{retries}...")
            time.sleep(delay)
    return None

In [5]:
def get_table(link):
    response = fetch_with_retry(link)
    response.text
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('div', id="dataflame").find('table',id="datatable")
    trs = table.find_all('tr')
    # Danh sách chứa tiêu đề
    headers = []
    rowspans = []  # Lưu trữ các cột có rowspan từ dòng trước

    for row in trs[:2]:
        cols = row.find_all("th")
        row_headers = []
        col_index = 0  # Chỉ số cột thực tế (sau khi tính colspan và rowspan)

        for col in cols:
            colspan = int(col.get("colspan", 1))
            rowspan = int(col.get("rowspan", 1))
            col_text = col.get_text(strip=True)

            # Nếu có rowspan từ dòng trước, thêm nó vào trước tiên
            while col_index < len(rowspans) and rowspans[col_index]:
                row_headers.append(rowspans[col_index])
                rowspans[col_index] = None  # Đánh dấu đã sử dụng
                col_index += 1

            # Thêm tiêu đề vào hàng hiện tại
            for _ in range(colspan):
                row_headers.append(col_text)
                rowspans.append(col_text if rowspan > 1 else None)  # Lưu lại nếu có rowspan
                col_index += 1

        # Nếu có rowspan từ dòng trước, cần tiếp tục thêm chúng vào
        while col_index < len(rowspans):
            if rowspans[col_index]:
                row_headers.append(rowspans[col_index])
            col_index += 1

        # Nếu là dòng đầu tiên, gán headers
        if not headers:
            headers = row_headers
        else:
            # Gộp tiêu đề với dòng trên nếu chưa đầy đủ
            headers = [h + " - " + r if r else h for h, r in zip(headers, row_headers)]

    data = []
    for row in trs[2:]:
        cols = row.find_all(["th", "td"])
        col_index = 0  # Chỉ số cột thực tế (sau khi tính colspan)
        row_data = []
        for col in cols:
            colspan = int(col.get("colspan", 1))
            col_text = col.get_text(strip=True)
            row_data.extend([col_text] * colspan)
            col_index += colspan
        data.append(row_data)
    df = pd.DataFrame(data,columns=headers)
    print(len(data))
    
    return df

In [ ]:
all_dfs = []  

for link in linklist:
    print("Processing index :"linklist.index(link))
    print("Processing :", link)
    df = get_table(link) 
    all_dfs.append(df) 

final_df = pd.concat(all_dfs, ignore_index=True)

final_df.to_excel('japan_tariff.xlsx',dtype=str)

0
Processing : https://www.customs.go.jp/english/tariff/2024_04_01/data/e_01.htm
81
1
Processing : https://www.customs.go.jp/english/tariff/2024_04_01/data/e_02.htm
195
2
Processing : https://www.customs.go.jp/english/tariff/2024_04_01/data/e_03.htm
675
3
Processing : https://www.customs.go.jp/english/tariff/2024_04_01/data/e_04.htm
353
4
Processing : https://www.customs.go.jp/english/tariff/2024_04_01/data/e_05.htm
56
5
Processing : https://www.customs.go.jp/english/tariff/2024_04_01/data/e_06.htm
28
6
Processing : https://www.customs.go.jp/english/tariff/2024_04_01/data/e_07.htm
224
7
Processing : https://www.customs.go.jp/english/tariff/2024_04_01/data/e_08.htm
178
8
Processing : https://www.customs.go.jp/english/tariff/2024_04_01/data/e_09.htm
144
9
Processing : https://www.customs.go.jp/english/tariff/2024_04_01/data/e_10.htm
107
10
Processing : https://www.customs.go.jp/english/tariff/2024_04_01/data/e_11.htm
171
11
Processing : https://www.customs.go.jp/english/tariff/2024_04_01